In [ ]:
"""
Purpose: To visualize how the proofreading performs on the manually proofread cells

Pseucode: 
For the example neuron id
1) get the neuron decomposition
2) Run the error detection to get the errored faces
3) Get the errored synapses
4) Plot them mesh and errored synapses



"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path
import neuron_visualizations as nviz


import datajoint_utils as du
from importlib import reload

INFO - 2020-11-30 17:59:57,904 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-30 17:59:57,905 - settings - Setting database.user to celiib
INFO - 2020-11-30 17:59:57,906 - settings - Setting database.password to newceliipass
INFO - 2020-11-30 17:59:57,911 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-30 17:59:57,912 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 17:59:57,925 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-30 17:59:58,127 - settings - Setting enable_python_native_blobs to True


In [2]:
import time
import datajoint_utils as du
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

from datajoint_utils import *

import error_detection as ed

minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-30 17:59:58,148 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-30 17:59:58,355 - settings - Setting enable_python_native_blobs to True


In [3]:
m65 = minnie
decimation_version = 0
decimation_ratio = 0.25
key_source = (m65.Decimation().proj(decimation_version='version')  &
              dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)
              & m65.ManualProofreadIDs() & (m65.BaylorSegmentCentroid() & "multiplicity=1").proj())
nucleus_table =(m65.NucleusID & 'nucleus_version=1')  # Aug 1 timestamp
proofread_nucs = nucleus_table & m65.ManualProofreadIDs()
single_soma_nucs = nucleus_table & (dj.U('segment_id').aggr(proofread_nucs, n_nucs='count(nucleus_id)') & 'n_nucs=1').proj()
final_nucs = key_source* single_soma_nucs
final_nucs

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),nucleus_id id of segmented nucleus,nucleus_version segmentation version,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),size (EM voxels) scaled by (4x4x40),svid supervoxel_id of centroid
864691135564655959,0,0.25,225498,1,149712,151648,17549,1315328,85369450370640387
864691135748568361,0,0.25,230236,1,162848,184560,19809,1385955,87203504753932217
864691136194042326,0,0.25,258307,1,174944,137648,21113,1587826,88815663879212610
864691135233108569,0,0.25,262773,1,168096,161952,21503,1314296,87904100020764569
864691135740225387,0,0.25,264870,1,164832,177856,21208,1831649,87413717834430502
864691136105498585,0,0.25,269247,1,177536,213824,19894,1571057,89177746601154881
864691134988386682,0,0.25,269380,1,168016,216288,21534,1688056,87911453071322695
864691136333776819,0,0.25,271518,1,168464,226192,21121,1666190,87912758674384086
864691135771629819,0,0.25,294545,1,182976,135728,21116,1462702,89941288908168255
864691135748575017,0,0.25,296726,1,180976,143888,22162,1827886,89660913577301286


In [4]:
eligible_table = minnie.Decomposition & final_nucs.proj()

In [5]:
seg_ids_to_test = eligible_table.fetch("segment_id")
seg_ids_to_test

array([864691135564655959, 864691135748568361, 864691136194042326,
       864691135233108569, 864691135740225387, 864691136105498585,
       864691134988386682, 864691136333776819, 864691135771629819,
       864691136105484249, 864691135212632448, 864691136296650011,
       864691136311740477, 864691135491233631, 864691135945413668,
       864691135393964789, 864691135113167769, 864691136105493209,
       864691135272164113, 864691136618403213, 864691135974454639,
       864691136003930314, 864691135510455760, 864691136537220258,
       864691135785277636, 864691136333787571, 864691136105491417,
       864691135645592260, 864691136056391384, 864691136099807093,
       864691135348272855, 864691135737374100, 864691135212690816,
       864691136550489250, 864691136194103510, 864691135589907979,
       864691135407289801, 864691136545546146, 864691135382947307,
       864691136008425132, 864691135925564174, 864691136333790899,
       864691135974454895, 864691136008573614, 864691136134446

# Doing the Actual Function

In [6]:
segment_id = seg_ids_to_test[4]
key = dict(segment_id=segment_id)
print(key)

{'segment_id': 864691135740225387}


In [7]:
segment_id = key["segment_id"]
global_start = time.time()

verbose = True

print(f"\n\n----- Working on {segment_id}-------")
whole_pass_time = time.time()

neuron_obj = (minnie.Decomposition() & key).fetch1("decomposition")
errored_faces = ed.error_faces_by_axons(neuron_obj,verbose=True,visualize_errors_at_end=False)

print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")



----- Working on 864691135740225387-------
Decompressing Neuron in minimal output mode...please wait
functions_list = [<function width at 0x7ff1519b27b8>, <function median_mesh_center at 0x7ff1519b2a60>, <function n_spines at 0x7ff1519b28c8>, <function n_faces_branch at 0x7ff153b1a488>, <function skeleton_distance_branch at 0x7ff1519b2840>, <function spines_per_skeletal_length at 0x7ff1519b2d08>, <function no_spine_median_mesh_center at 0x7ff1519b2b70>]
functions_list = [<function axon_segment at 0x7ff1519b3400>]
Axons not keeping because of soma: {'L1': array([ 6,  7,  8,  9, 12, 13])}

----- Working on L0 ------
-- Axon Group 0 of size 29--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- Axon Group 1 of size 1--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- Axon Group 2 of size 1--
   Working on soma 0, starting_node 4
Not using AIS angle threshold
sk_angle= 8.215939908057189
-- 

# Visualize the Actual Neuron

In [8]:
print(neuron_obj.segment_id)
nviz.plot_objects(main_mesh=neuron_obj.mesh,
             mesh_alpha=1)

864691135740225387


# Visualize the Results

In [ ]:
print(f"n_errored_faces = {len(errored_faces)}")

In [ ]:
ed = reload(ed)


if neuron_obj is None:
    neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")

current_mesh = neuron_obj.mesh

error_submesh = current_mesh.submesh([errored_faces],append=True)
valid_mesh = tu.subtract_mesh(current_mesh,error_submesh)

In [ ]:
nviz.plot_objects(main_mesh=valid_mesh,
             meshes=error_submesh,
              meshes_colors="red",
             mesh_alpha=1)

# How to get the percentage of synapses removed

In [ ]:
"""
Pseudocode: 
1) Get the synapse file that corresponds to it
2) Get the matching v7 id
3) Get the v7 synapses (pre and post) that are still there
4) Get the pre and post that are stil their after autoproofreading
5) Things to find:
- Eliminated and should have been (true positives)
- Eliminated but SHOULDN'T (false positives) 
- Not Eliminated and should have been not (true negative)
- Not Eliminated but should have been eliminated (false negatie) 

"""

In [ ]:
# 1) Get the synapse file that corresponds to it
data_path = Path("./manual_proofread_data/")
current_file = data_path / Path(f"{segment_id}.pkl")
current_file

In [ ]:
import system_utils as su
curr_synapse_table = su.load_object(current_file)
#eliminating the self connections
df = curr_synapse_table[curr_synapse_table["pre_pt_root_id_Aug"] != curr_synapse_table["post_pt_root_id_Aug"]]
df

In [ ]:
#1b) Get the synapse data for the august neuron 
ctr_pt_position = df["ctr_pt_position"].to_numpy()
synapse_ids = df["id"].to_numpy()
synapse_centers = np.vstack(ctr_pt_position)
synapse_ids.shape,synapse_centers.shape
timestamps = np.zeros(len(synapse_ids))

In [ ]:
# 2) Get the matching v7 id
import pandas as pd
aug_to_v7_table = pd.read_csv("./proofread_cells_seg_correspondence_Aug1-v7.csv")
v7_id = aug_to_v7_table[aug_to_v7_table["old_seg_id"]==segment_id]["new_seg_id"].tolist()[0]
v7_id

In [ ]:
# check that the new v7 segment id is in the synapse table

def get_pd_id_data(col_name,return_unique=False):
    ret_data = df [col_name].to_numpy()
    if return_unique:
        return np.unique(ret_data)
    else:
        return ret_data
    
pre_pt_root_id = get_pd_id_data("pre_pt_root_id")
pre_pt_root_id.shape,v7_id in pre_pt_root_id

In [ ]:
post_pt_root_id = get_pd_id_data("post_pt_root_id")
post_pt_root_id.shape,v7_id in post_pt_root_id

In [ ]:
original_presyn = df[(df["pre_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()
original_postsyn = df[(df["post_pt_root_id_Aug"] == segment_id)]["id"].to_numpy()

original_presyn.shape,original_postsyn.shape

In [ ]:
#3) Get the v7 synapses (pre and post) that are still there
manual_remaining_presyn = df[(df["pre_pt_root_id"] == v7_id)]["id"].to_numpy()
manual_remaining_postsyn = df[(df["post_pt_root_id"] == v7_id)]["id"].to_numpy()
manual_remaining_presyn.shape,manual_remaining_postsyn.shape

In [ ]:
# find the non-remaining presyns and postsyns
manual_error_presyn = np.setdiff1d(original_presyn,manual_remaining_presyn)
manual_error_postsyn = np.setdiff1d(original_postsyn,manual_remaining_postsyn)

manual_error_presyn.shape,manual_error_postsyn.shape

In [ ]:
len(manual_remaining_presyn) + len(manual_error_presyn), len(manual_remaining_postsyn) + len(manual_error_postsyn)

In [ ]:
#4) Get the pre and post that are stil their after autoproofreading
ed = reload(ed)
err_synapse_ids,non_err_synapse_ids = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                     errored_faces,return_synapse_ids=True,
                                                                   synapse_centers=synapse_centers,
                                                                   synapse_ids=synapse_ids
                                                                  )

In [ ]:
# divided the valid synapses into presyn and postsyn
auto_remaining_presyn = np.intersect1d(original_presyn,non_err_synapse_ids)
auto_remaining_postsyn = np.intersect1d(original_postsyn,non_err_synapse_ids)

#dividing the error synapses into presyn and postsyn
auto_error_presyn = np.intersect1d(original_presyn,err_synapse_ids)
auto_error_postsyn = np.intersect1d(original_postsyn,err_synapse_ids)

len(auto_remaining_presyn), len(auto_error_presyn), len(auto_remaining_postsyn), len(auto_error_postsyn)

# Calculating the final scores

In [ ]:
# doing the presyn analysis
true_positive_presyn = np.intersect1d(manual_error_presyn,auto_error_presyn) #deemed an error and was an error
false_positive_presyn = np.intersect1d(manual_remaining_presyn,auto_error_presyn)#deemed an error but not an error

true_negative_presyn = np.intersect1d(manual_remaining_presyn,auto_remaining_presyn) #not deemed an error and should not be an error
false_negative_presyn = np.intersect1d(manual_error_presyn,auto_remaining_presyn) #not deemed and error but should be an error

len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)

In [ ]:
np.sum([len(true_positive_presyn),len(false_positive_presyn),len(true_negative_presyn),len(false_negative_presyn)])

In [ ]:
# Doing the postsyn analysis
true_positive_postsyn = np.intersect1d(manual_error_postsyn,auto_error_postsyn) #deemed an error and was an error
false_positive_postsyn = np.intersect1d(manual_remaining_postsyn,auto_error_postsyn)#deemed an error but not an error

true_negative_postsyn = np.intersect1d(manual_remaining_postsyn,auto_remaining_postsyn) #not deemed an error and should not be an error
false_negative_postsyn= np.intersect1d(manual_error_postsyn,auto_remaining_postsyn) #not deemed and error but should be an error

len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)

In [ ]:
np.sum([len(true_positive_postsyn),len(false_positive_postsyn),len(true_negative_postsyn),len(false_negative_postsyn)])

In [ ]:
#
true_positive_total = list(true_positive_presyn) + list(true_positive_postsyn) 
false_positive_total = list(false_positive_presyn) + list(false_positive_postsyn)

true_negative_total = list(true_negative_presyn) + list(true_negative_postsyn)
false_negative_total = list(false_negative_presyn) + list(false_negative_postsyn)

len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)

In [ ]:
np.sum([len(true_positive_total),len(false_positive_total),len(true_negative_total),len(false_negative_total)])

In [ ]:
def precision(TP,FP):
    if (TP + FP) > 0:
        return TP/(TP + FP)
    else:
        return np.nan
    
def recall(TP,FN):
    if (TP + FN) > 0:
        return TP/(TP + FN)
    else:
        return np.nan
    
def f1(TP,FP,FN):
    curr_prec = precision(TP,FP)
    curr_recall = recall(TP,FN)
    
    if curr_prec + curr_recall > 0:
        return 2*(curr_prec*curr_recall)/(curr_prec + curr_recall)
    else:
        return np.nan
    
def calculate_scores(TP,FP,FN):
    return dict(precision=precision(TP,FP),
               recall=recall(TP,FN),
               f1=f1(TP,FP,FN))

In [ ]:
# calculating the accuracy, precision and f1

#precision = (true positives)/(true positives + false positives)
presyn_scores =  calculate_scores(len(true_positive_presyn),len(false_positive_presyn),len(false_negative_presyn))
presyn_scores

In [ ]:
postsyn_scores =  calculate_scores(len(true_positive_postsyn),len(false_positive_postsyn),len(false_negative_postsyn))
postsyn_scores

In [ ]:
total_scores =  calculate_scores(len(true_positive_total),len(false_positive_total),len(false_negative_total))
total_scores

# If no automatic proofreading was done

# Visualize the Results

In [ ]:

err_synapses,non_err_synapses = ed.get_error_synapse_inserts(current_mesh,segment_id,
                                                     errored_faces,return_synapse_centroids=True,
                                                                   synapse_centers=synapse_centers,
                                                                   synapse_ids=synapse_ids
                                                                  )

print(f"Segment ID = {segment_id}")
print(f"n_synapses = {len(err_synapses) + len(non_err_synapses)}, n_errored_synapse = {len(err_synapses)}")


valid_synapse_color = "yellow"
error_color = "red"
scatters_list = []
scatters_colors = []

if len(err_synapses) > 0:
    scatters_list.append(err_synapses)
    scatters_colors.append(error_color)
if len(non_err_synapses) > 0:
    scatters_list.append(non_err_synapses)
    scatters_colors.append(valid_synapse_color)
    
if len(scatters_list) > 0:
    nviz.plot_objects(main_mesh=valid_mesh,
              meshes=[error_submesh],
                      mesh_alpha=1,
              meshes_colors=[error_color],
            scatters=[err_synapses,non_err_synapses],
             scatters_colors=[error_color,valid_synapse_color],
                     scatter_size=0.2)

else:
    nviz.plot_objects(main_mesh=valid_mesh,
             meshes=error_submesh,
              meshes_colors=error_color,
             mesh_alpha=1)


# Write the Results to a Datajoint Table

In [ ]:
@schema
class ValidationStats(dj.Manual):
    definition="""
    -> minnie.ManualProofreadIDs
    ---
    new_segment_id: bigint unsigned
    
    true_positive_presyn: int
    false_positive_presyn: int
    true_negative_presyn: int
    false_negative_presyn: int

    precision_presyn=NULL: double
    recall_presyn=NULL: double
    f1_presyn=NULL: double

    true_positive_postsyn: int
    false_positive_postsyn: int
    true_negative_postsyn: int
    false_negative_postsyn: int

    precision_postsyn=NULL: double
    recall_postsyn=NULL: double
    f1_postsyn=NULL: double

    true_positive_total : int
    false_positive_total : int
    true_negative_total : int
    false_negative_total : int

    precision_total=NULL: double
    recall_total=NULL: double
    f1_total=NULL: double

    """

In [ ]:
info_key = dict(segment_id=segment_id,
               new_segment_id=v7_id,
               
                true_positive_presyn = len(true_positive_presyn),
                false_positive_presyn = len(false_positive_presyn),
                true_negative_presyn= len(true_negative_presyn),
                false_negative_presyn = len(false_negative_presyn),

                precision_presyn = presyn_scores["precision"],
                recall_presyn = presyn_scores["recall"],
                f1_presyn = presyn_scores["f1"],

                true_positive_postsyn = len(true_positive_postsyn),
                false_positive_postsyn = len(false_positive_postsyn),
                true_negative_postsyn = len(true_negative_postsyn),
                false_negative_postsyn = len(false_negative_postsyn),

                precision_postsyn = postsyn_scores["precision"],
                recall_postsyn = postsyn_scores["recall"],
                f1_postsyn = postsyn_scores["f1"],

                true_positive_total= len(true_positive_total),
                false_positive_total = len(false_positive_total),
                true_negative_total = len(true_negative_total),
                false_negative_total = len(false_negative_total),

                precision_total = total_scores["precision"],
                recall_total=total_scores["recall"],
                f1_total = total_scores["f1"],
                
                
               )

ValidationStats.insert1(info_key,skip_duplicates=True)

In [ ]:
ValidationStats()

In [ ]:
TP,FP,FN = ValidationStats.fetch("true_positive_total","false_positive_total","false_negative_total")
total_dict = dict(type="presyn and postsyn")
total_stats = calculate_scores(TP = np.sum(TP),FP = np.sum(FP),FN = np.sum(FN))
total_dict.update(total_stats)
total_dict

In [ ]:
TP_pre,FP_pre,FN_pre = ValidationStats.fetch("true_positive_presyn","false_positive_presyn","false_negative_presyn")
presyn_dict = dict(type="presyn")
presyn_stats = calculate_scores(TP = np.sum(TP_pre),FP = np.sum(FP_pre),FN = np.sum(FN_pre))
presyn_dict.update(presyn_stats)
presyn_dict

In [ ]:
TP_post,FP_post,FN_post = ValidationStats.fetch("true_positive_postsyn","false_positive_postsyn","false_negative_postsyn")
postsyn_dict = dict(type="postsyn")
postsyn_stats = calculate_scores(TP = np.sum(TP_post),FP = np.sum(FP_post),FN = np.sum(FN_post))
postsyn_dict.update(postsyn_stats)
postsyn_dict

# Making the Figure for the Stats

In [ ]:
import pandas as pd

In [ ]:
pd = 